# Introduction to PyTorch

**Nicolas Nytko**  
nnytko2@illinois.edu  
https://github.com/nicknytko/siam-pytorch-workshop

Adapted from materials by Matthew West (mwest@illinois.edu)

_SIAM @ Illinois workshop series_

November 12, 2024

**Description:** PyTorch allows you to easily train and run machine learning models. It uses standard Python methods for writing code, so it's both simple and powerful. We will cover the core automatic differentiation capabilities of PyTorch, training deep neural networks, managing training and test data, saving and loading models, and show a few examples of neural network implementations. We will assume a good knowledge of Python and NumPy, and basic knowledge of machine learning with neural nets.

# List of resources

- PyTorch tutorials: https://pytorch.org/tutorials/
- PyTorch manual: https://pytorch.org/docs/stable/index.html
- PyTorch paper: https://openreview.net/forum?id=BJJsrmfCZ
- Calculus in Machine Learning: https://machinelearningmastery.com/calculus-in-machine-learning-why-it-works/
- Calculus on computational graphs: http://colah.github.io/posts/2015-08-Backprop/
- Einstein summation in PyTorch: https://rockt.github.io/2018/04/30/einsum

# First, some math background

Three main "approaches" to machine learning:
- Supervised learning
- Unsupervised learning
- Reinforcement learning

We'll focus on supervised learning, though what we'll show here will be applicable to all 3.

Let's say we have some unknown function $f : \mathbb{R}^m \to \mathbb{R}^n$; for example, a function that takes an image and outputs $1$ if it contains a cat, and $0$ otherwise.

![](figures/f_catdog.png)

<small> Dog photo from https://www.pickpik.com/dog-puppy-cute-adorable-pet-cute-puppy-59501 </small>

This is pretty easy for humans to determine!  However, we don't know what such a function "looks like" nor how we would implement it on a computer.  So, we seek an approximate function $\hat{f} \approx f$ where $\hat{f}$ is much easier for us to evaluate.

Here's a smaller example with a linear regression.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 1, 50)
f = 5 + 3*x + np.random.randn(50)*0.1

plt.plot(x, f, 'o')

Here, the output of $f$ is displayed for a few values of $x$.  This is pretty simple, but useful for demonstration!  Let us try to fit a $\hat{f}$ that approximates $f$ closely.  Lets assume $\hat{f}$ is a linear function of the form
$$ \hat{f}(x; m, b) = mx + b. $$
One way of forcing $\hat{f} \approx f$ is to minimize the function
$$ \ell(m, b) = \sum_i (f(x_i, m, b) - f(x))^2 = \sum_i (mx_i + b - f(x))^2 $$
How do we do this?  Recall from calculus that to find the minimum of a function, we can take the derivatives and solve for $0$.

$$\begin{align*}
\frac{\partial \ell}{\partial m} &= \sum_i 2x_i(mx_i + b - f(x)) \\
\frac{\partial \ell}{\partial b} &= \sum_i 2(mx_i + b - f(x))
\end{align*}$$

Numerically, we can implement this using *gradient descent*.  We start with some random initial values of $m, b$ and update like
$$\begin{align*}
m &\gets m - \alpha \frac{\partial \ell}{\partial m}(m, b), \\
b &\gets b - \alpha \frac{\partial \ell}{\partial b}(m, b),
\end{align*}$$
where $\alpha \in \mathbb{R}^+$ is some small parameter called the *learning rate*.  Lets see it in action:

In [ ]:
# Start with an arbitrary guess for the slope and bias
m = 1.0
b = 0.0

alpha = 0.01

In [ ]:
# Compute derivatives
dl_dm = np.sum(2 * x * (m * x + b - f))
dl_db = np.sum(2 * (m * x + b - f))

# Descend
m -= dl_dm * alpha
b -= dl_db * alpha

# Plot the results
f_hat = m * x + b

plt.plot(x, f, 'o')
plt.plot(x, f_hat)
plt.ylim(np.min(f) - 1, np.max(f) + 1)

print(f'Loss: {np.sum((f_hat - f) ** 2):.3f}')
print(f'Derivatives: {dl_dm:.3e} {dl_db:.3e}')
print(f'm: {m:.3f}, b: {b:.3f}')

This is the basic idea behind supervised learning!  Of course, we did everything by hand, like taking the derivatives and performing gradient descent, which is a lot of work as our models will get more complex.  As we'll see, PyTorch can implement much of this for us.

# Let's get started with PyTorch!

In [ ]:
import torch
import torch.linalg as tla
import random, datetime

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0])

In [ ]:
x = torch.Tensor([1.0, 2.0, 3.0])

In [ ]:
x

In [ ]:
x ** 2

In [ ]:
torch.cos(x)

In [ ]:
tla.norm(x)

In [ ]:
x**2

In [ ]:
x.dtype

#### Specify datatypes

Use `dtype=torch.float64` and `.double()`

In [ ]:
y = x.double()
y

In [ ]:
y.dtype

In [ ]:
y.float()

In [ ]:
y.int()

In [ ]:
x = torch.tensor([1, 2, 3])

In [ ]:
torch.log(x)

In [ ]:
x.dtype

In [ ]:
x = torch.tensor([1, 2, 3], dtype=torch.float64)

In [ ]:
torch.log(x)

#### Annoying differences from NumPy

`np.sum(x, axis=1)` versus `torch.sum(x, dim=1)`

In [ ]:
T = torch.Tensor([[1,2,3], [4,5,6]])
Tary = np.array([[1,2,3], [4,5,6]])

In [ ]:
np.sum(Tary, axis=1)

In [ ]:
torch.sum(T, dim=1)

#### Convert to/from NumPy arrays

`.numpy()` and `torch.from_numpy()`

In [ ]:
x = np.array([1, 2, 3])
y = torch.from_numpy(x)
y

In [ ]:
y.numpy()

Memory is shared!

In [ ]:
x[0] = 7
y

# `torch.autograd`: Computing derivatives

PyTorch constructs the computation graph as you do operations (dynamic graphs) unlike TensorFlow (static graphs)

Using the computation graph, the chain rule (back propagation) can compute derivatives

Derivatives are available in the leaf nodes

![](figures/computation_graph.jpg)

<small>Figure from http://datahacker.rs/004-computational-graph-and-autograd-with-pytorch/</small>

In [ ]:
x = torch.tensor(5.0)

In [ ]:
y = torch.tensor(3.0, requires_grad=True)

In [ ]:
z = x * y**2
z

In [ ]:
z.backward()

In [ ]:
print(f'x.grad = {x.grad}')

In [ ]:
y.grad

$z = x y^2$

$\frac{\partial z}{\partial y} = 2 x y$

In [ ]:
2*x*y

#### Control what we differentiate with respect to

`requires_grad=True`

`with no_grad():`

`.detach()`

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = x*x
print(f'y.requires_grad = {y.requires_grad}')
z = x*y
z.backward()
print(f'dz/dx = {x.grad}')

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = x*x
y = y.detach() # can't say y.requires_grad = False
print(f'y.requires_grad = {y.requires_grad}')
z = x*y
z.backward()
print(f'dz/dx = {x.grad}')

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
with torch.no_grad():
    y = x*x
print(f'y.requires_grad = {y.requires_grad}')
z = x*y
z.backward()
print(f'dz/dx = {x.grad}')

#### Computation graphs are not trees

Re-using a parameter in multiple places makes the graph not be a tree. It's a DAG.

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = 3*x
z = x**2
w = y + z + x
w.backward()
x.grad

$\frac{\partial w}{\partial x} = \frac{\partial}{\partial x}(3x + x^2 + x) = 3 + 2x + 1$

In [ ]:
3 + 2*x + 1

#### The computation graph is destroyed by `backward()`

To retain it for more differentiation, use `backward(retain_graph=True)`

A common use case is multiple outputs with a shared subgraph

Don't forget to free the graph on the last call to prevent memory leaks

In [ ]:
x = torch.tensor(3.0, requires_grad=True)
y = x**2
z1 = 3*y
z2 = 4*y

In [ ]:
z1.backward() # (retain_graph=True)
x.grad

In [ ]:
z2.backward()
x.grad

#### Derivatives of scalars with respect to tensors

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = (x**2).sum()
y.backward()
x.grad

#### Don't do in-place modifications to tensors

But it's fine to do `x = 4 * x`

In [ ]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
x

In [ ]:
x[1] = x[2] + 1
#x = 4*x
x

In [ ]:
y = (x**2).sum()
y.backward()

#### Results can be slightly different from what you expect...

Since we're building the graph as computations are being done, functions like `max()` become differentiable

In [ ]:
x = torch.tensor([1.0, 2.0, 4.0, 3.0, 0.5], requires_grad=True)
max_x = torch.max(x)
max_x

In [ ]:
max_x.backward()
x.grad

# `torch.optim`: All the common gradient-based optimizers

In [ ]:
def f(x):
    return x**2

In [ ]:
xvec = np.linspace(-2, 2, 100)
fvec = f(xvec)
plt.plot(xvec, fvec, 'o-', markersize=3)

In [ ]:
x = torch.tensor([2.0], requires_grad=True)

In [ ]:
opt = torch.optim.SGD([x], lr=0.1)

In [ ]:
x_history = [x.detach().numpy().copy()]
for i in range(30):
    print(f'##########')
    print(f'i = {i}')
    print(f'initial x = {x}')
    opt.zero_grad()
    z = f(x)
    print(f'f(x) = {z}')
    z.backward()
    print(f'x.grad = {x.grad}')
    opt.step()
    print(f'updated x = {x}')
    x_history.append(x.detach().numpy().copy())

In [ ]:
xvec = np.linspace(-2, 2, 100)
fvec = f(xvec)
plt.plot(xvec, fvec)
plt.plot(x_history, f(np.array(x_history)), 'o-')

# `torch.nn`: Lets implement a neural network!

Recall from the first example, we implemented a linear model to approximate some noisy data.

What happens if we compose multiple linear models together? 

$$\begin{align*}
f_1{\bf x} &= {\bf W}_1 {\bf x} + {\bf b}_1 \\
f_2{\bf x} &= {\bf W}_2 {\bf x} + {\bf b}_2 \\
\hat{f}({\bf x}) &= f_2(f_1({\bf x})) = {\bf W}_2({\bf W}_1 {\bf x} + {\bf b}_1) + {\bf b}_2
\end{align*}$$

If we distribute terms out, we get something that's still linear in ${\bf x}$!  
$$ \hat{f}({\bf x}) = {\bf W}_2({\bf W}_1 {\bf x} + {\bf b}_1) + {\bf b}_2 = {\bf W}_2 {\bf W}_1 {\bf x} + {\bf W}_2 {\bf b}_1 + {\bf b}_2 $$

This means that ultimately stacking many linear models together does not give us something more expressive.  However, we can introduce a nonlinearity in terms of an *activation function*.  There are many choices, like

$$\begin{align*}
\text{ReLU}({\bf x})_i &= \max\{x_i, 0\} \\
\sigma({\bf x})_i &= e^x/(1 + e^x) \\
\text{Softplus}({\bf x})_i &= \ln(1+e^x) \\
&\enspace\vdots
\end{align*}$$

and many more.  We'll stick to ReLU here as it's pretty simple.  Adding these nonlinear functions to our model, we get
$$\begin{align*}
f_1{\bf x} &= \text{ReLU}({\bf W}_1 {\bf x} + {\bf b}_1) \\
f_2{\bf x} &= \text{ReLU}({\bf W}_2 {\bf x} + {\bf b}_2) \\
\hat{f}({\bf x}) &= f_2(f_1({\bf x})) = \text{ReLU}({\bf W}_2\text{ReLU}({\bf W}_1 {\bf x} + {\bf b}_1) + {\bf b}_2)
\end{align*}$$
Notice this is no longer linear in ${\bf x}$.  Lets go ahead and implement this in PyTorch.

In [ ]:
x = torch.linspace(0, 2*np.pi, 100)
y = torch.sin(x)
plt.plot(x.numpy(), y.numpy(), 'r.');

In [ ]:
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        N = 8
        self.fc1 = torch.nn.Linear(1, N)
        self.fc2 = torch.nn.Linear(N, N)
        self.fc3 = torch.nn.Linear(N, 1)

    def forward(self, x):
        x = x.reshape((-1, 1)) # We'll reshape this to a column vector
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x.flatten()

In [ ]:
model = MyModel()

In [ ]:
yp = model(x)

plt.plot(x.numpy(), y.numpy(), 'r.');
plt.plot(x.numpy(), yp.detach().numpy());

In [ ]:
model.fc3.weight

In [ ]:
model.fc3.bias

In [ ]:
model.fc3.bias.data

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.01)
loss_history = []

In [ ]:
for i in range(200):
    opt.zero_grad()
    
    yp = model(x)
    loss = torch.nn.MSELoss()(yp, y)
    
    loss_history.append(loss.item())
    loss.backward()
    
    opt.step()

plt.plot(loss_history)
plt.xlabel('Training iteration')
plt.ylabel('Loss')
plt.grid()

In [ ]:
plt.plot(x.numpy(), y.numpy(), 'r.');
plt.plot(x.numpy(), yp.detach().numpy());

# Saving and restoring models

Save and load the parameters, not the full models.

In [ ]:
torch.save(model.state_dict(), 'model_file.pth')

In [ ]:
model = MyModel()
model.load_state_dict(torch.load('model_file.pth', weights_only=True))

In [ ]:
model.state_dict()